# Building Footprints with Census Geography Join

This notebook performs spatial joins between Microsoft Global ML Building Footprints and US Census TIGER/Line geographies to enrich building data with census block and place information.

## Features

* **Spatial joins** - Joins building footprints with census blocks and places using geospatial intersections
* **Deduplication logic** - When a building intersects multiple geographies, assigns it to the one with the largest intersection area
* **Geometry validation** - Uses `st_buffer(geom, 0)` to fix invalid geometries before spatial operations
* **WKB format support** - Efficiently reads geometries stored in Well-Known Binary format
* **Parameterized** - Ready for job/pipeline execution with configurable census year

## Parameters

| Parameter | Description | Example |
|-----------|-------------|----------|
| `tiger_year` | Census TIGER/Line year for output table naming | `2020` |

**Note**: The `tiger_year` parameter is used only for output table naming convention. The notebook processes all data from the bronze tables without year filtering.

## Input Tables

| Table | Description |
|-------|-------------|
| `odi_datalake.odi_bronze.global_ml_building_footprints_california` | Microsoft ML building footprints with WKB geometries |
| `odi_datalake.odi_bronze.tiger_blocks` | Census blocks with WKB geometries |
| `odi_datalake.odi_bronze.tiger_places` | Census places (cities, towns) with WKB geometries |

## Output Table

**Table**: `odi_datalake.odi_silver.building_footprints_with_blocks_and_places_{tiger_year}`

**Schema**:
* `height` - Building height estimate (meters)
* `geometry` - Building footprint geometry
* `area_sqm` - Building footprint area (square meters)
* `county_fips` - County FIPS code
* `tract` - Census tract code
* `block` - Census block code
* `block_geoid` - Full census block GEOID
* `place_fips` - Place FIPS code (nullable)
* `place_ns` - Place ANSI code (nullable)
* `place_geoid` - Place GEOID (nullable)
* `place_name` - Place name (nullable)
* `class_fips_code` - Place classification code (nullable)
* `class_fips` - Place classification (nullable)

## How It Works

1. **Load building footprints** - Converts WKB geometries to spatial objects
2. **Load census blocks** - Converts WKB geometries to spatial objects
3. **Spatial join with blocks** - Joins footprints with blocks using `st_intersects`
   - Calculates intersection area for each footprint-block pair
   - Deduplicates by keeping the block with largest intersection per footprint
4. **Load census places** - Converts WKB geometries to spatial objects
5. **Spatial join with places** - Left joins footprints with places using `st_intersects`
   - Calculates intersection area for each footprint-place pair
   - Deduplicates by keeping the place with largest intersection per footprint
   - Uses left join so footprints outside incorporated places are retained
6. **Calculate area** - Computes building footprint area in square meters
7. **Save results** - Writes enriched data to Delta table in silver layer

## Performance Optimizations

* **PySpark over SQL** - Uses PySpark DataFrames for spatial joins to compute intersection area once and reuse it
* **Geometry validation** - Applies `st_buffer(geom, 0)` to fix invalid geometries before spatial operations
* **Window functions** - Efficiently deduplicates using window functions with `row_number()`
* **Temporary views** - Uses temporary views to organize multi-step transformations

## Notes

* Buildings that don't intersect any census place will have null place attributes
* Deduplication ensures each building is assigned to exactly one block and at most one place
* All geometries are assumed to be in the same coordinate reference system (CRS)
* The notebook uses serverless compute optimized for geospatial operations

In [0]:
# Import PySpark and Databricks SQL functions for geospatial operations
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.databricks.sql import functions as dbf

In [0]:
%sql
-- Load building footprints and convert WKT to geometry
CREATE OR REPLACE TEMPORARY VIEW footprints AS
SELECT 
  height,
  st_geomfromwkb(geometry_wkb) AS geometry
FROM odi_datalake.odi_bronze.global_ml_building_footprints_california;

-- Show count and sample
SELECT COUNT(*) AS footprint_count FROM footprints;

In [0]:
# Load census blocks and convert WKB to geometry

blocks = spark.table("odi_datalake.odi_bronze.tiger_blocks").select(
    F.col("COUNTYFP20").alias("county_fips"),
    F.col("TRACTCE20").alias("tract"),
    F.col("BLOCKCE20").alias("block"),
    F.col("GEOID20").alias("block_geoid"),
    dbf.st_geomfromwkb(F.col("geometry_wkb")).alias("geometry")
)

blocks.createOrReplaceTempView("blocks")

print(f"Block count: {blocks.count():,}")

In [0]:
# Load census places and convert WKB to geometry

places = spark.table("odi_datalake.odi_bronze.tiger_places").select(
    F.col("PLACEFP").alias("place_fips"),
    F.col("PLACENS").alias("place_ns"),
    F.col("GEOID").alias("place_geoid"),
    F.col("NAME").alias("place_name"),
    F.col("CLASSFP").alias("class_fips_code"),
    F.col("CLASSFP").alias("class_fips"),
    dbf.st_geomfromwkb(F.col("geometry_wkb")).alias("geometry")
)

places.createOrReplaceTempView("places")

print(f"Place count: {places.count():,}")

In [0]:
# Using PySpark instead of Spark SQL for this step because:
# 1. We can compute st_intersection once and reuse it (SQL would compute it 4x per row)
# 2. Better performance through caching and avoiding redundant calculations

# Load the temporary views as DataFrames
footprints_df = spark.table("footprints")
blocks_df = spark.table("blocks")

# Perform spatial join
footprints_blocks_joined = footprints_df.alias("f").join(
    blocks_df.alias("b"),
    dbf.st_intersects(F.col("f.geometry"), F.col("b.geometry")),
    "inner"
).select(
    F.col("f.height"),
    F.col("f.geometry"),
    F.col("b.county_fips"),
    F.col("b.tract"),
    F.col("b.block"),
    F.col("b.block_geoid"),
    # Calculate intersection area once and store it
    # Using st_buffer(geom, 0) to fix invalid geometries before intersection
    dbf.st_area(
        dbf.st_buffer(
            dbf.st_intersection(
                dbf.st_buffer(F.col("f.geometry"), F.lit(0)),
                dbf.st_buffer(F.col("b.geometry"), F.lit(0))
            ),
            F.lit(0)
        )
    ).alias("intersection_area")
)

# Deduplicate: keep only the block with largest intersection per footprint
window_spec = Window.partitionBy(dbf.st_astext(F.col("geometry"))).orderBy(F.desc("intersection_area"))

footprints_with_blocks = footprints_blocks_joined.withColumn(
    "row_num",
    F.row_number().over(window_spec)
).filter(
    F.col("row_num") == 1
).drop("row_num", "intersection_area")

# Create temporary view for next step
footprints_with_blocks.createOrReplaceTempView("footprints_with_blocks")

print(f"Footprints with blocks count: {footprints_with_blocks.count():,}")
display(footprints_with_blocks.limit(5))

In [0]:
# Using PySpark for consistency with the blocks join and for the same performance benefits
# (compute intersection once, better optimization, st_buffer for geometry validation)

# Load the temporary views as DataFrames
footprints_with_blocks_df = spark.table("footprints_with_blocks")
places_df = spark.table("places")

# Perform left spatial join
footprints_blocks_places_joined = footprints_with_blocks_df.alias("fb").join(
    places_df.alias("p"),
    dbf.st_intersects(F.col("fb.geometry"), F.col("p.geometry")),
    "left"
).select(
    F.col("fb.height"),
    F.col("fb.geometry"),
    F.col("fb.county_fips"),
    F.col("fb.tract"),
    F.col("fb.block"),
    F.col("fb.block_geoid"),
    F.col("p.place_fips"),
    F.col("p.place_ns"),
    F.col("p.place_geoid"),
    F.col("p.place_name"),
    F.col("p.class_fips_code"),
    F.col("p.class_fips"),
    # Calculate intersection area once and store it
    # Using st_buffer(geom, 0) to fix invalid geometries before intersection
    dbf.st_area(
        dbf.st_buffer(
            dbf.st_intersection(
                dbf.st_buffer(F.col("fb.geometry"), F.lit(0)),
                dbf.st_buffer(F.col("p.geometry"), F.lit(0))
            ),
            F.lit(0)
        )
    ).alias("intersection_area")
)

# Deduplicate: keep only the place with largest intersection per footprint
window_spec = Window.partitionBy(
    dbf.st_astext(F.col("geometry")), 
    F.col("block_geoid")
).orderBy(F.desc("intersection_area"))

footprints_with_blocks_and_places = footprints_blocks_places_joined.withColumn(
    "row_num",
    F.row_number().over(window_spec)
).filter(
    F.col("row_num") == 1
).drop("row_num", "intersection_area")

# Create temporary view for next step
footprints_with_blocks_and_places.createOrReplaceTempView("footprints_with_blocks_and_places")

In [0]:
%sql
-- Calculate area in square meters and create final view
CREATE OR REPLACE TEMPORARY VIEW footprints_final AS
SELECT 
  *,
  st_area(geometry) AS area_sqm
FROM footprints_with_blocks_and_places;

In [0]:
# Save results to a Delta table
# Using Python instead of SQL to dynamically construct table name with parameter

tiger_year = dbutils.widgets.get("tiger_year")
table_name = f"odi_datalake.odi_silver.building_footprints_with_blocks_and_places_{tiger_year}"

spark.sql(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM footprints_final")

print(f"Results saved to {table_name}")